In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from glob import glob
import shutil
import random

import csv

In [2]:
device_dict={"S":"Smartphone", 
             "T":"Tablet",
             "L":"Laptop",
             "V":"VehicleLCD",
             "M":"Monitor"}
status_dict={"F":"Focus"}

In [3]:
class Scenario():
    def __init__(self):
        self.scenarios={'F':[self.s0,self.s1],
                        'S':[self.s4],
                        'D':[self.s5,self.s6],
                        'A':[self.s4],
                        'N':[self.s2,self.s3]}
        
    def set_points(self, nframes):
        self.nframes = nframes
        self.x0 = np.linspace(0, 1920, self.nframes+2)[1:-1]
        self.y0 = np.linspace(0, 1080, self.nframes+2)[1:-1]
        
        self.xround = np.concatenate((self.x0[::2],self.x0[-1::-2]))
        self.yround = np.concatenate((self.y0[::2],self.y0[-1::-2]))

    def xrand(self):
        return np.repeat(480 + 960*np.random.rand(), self.nframes)
        
    def yrand(self):
        return np.repeat(270 + 540*np.random.rand(), self.nframes)

        
    def xrand2(self):
        return np.repeat(480*np.random.rand() + \
                         int(np.random.rand())*480*3,
                         self.nframes)
    def yrand2(self):
        return np.repeat(270*np.random.rand() + \
                         int(np.random.rand())*270*3,
                         self.nframes)

    def s0(self): 
        return [self.x0,self.y0]
    def s1(self): 
        return [np.repeat(30, self.nframes), self.y0]
    def s2(self): 
        return [self.xround, self.y0]
    def s3(self): 
        return[self.x0, self.yround]
    def s4(self):
        return [self.xrand(), self.yrand()]

    def s5(self): 
        return [np.concatenate((self.xrand()[::5],self.xrand2()[::5],
                          self.xrand()[::5],self.xrand2()[::5],
                          self.xrand()[::5])), self.yrand2()]
    def s6(self):
        return [np.concatenate((self.yrand()[::5],self.yrand2()[::5],
                          self.yrand()[::5],self.yrand2()[::5],
                          self.yrand()[::5])), self.xrand2()]

In [4]:
from scipy.interpolate import interp1d

In [5]:
device_dict={"S":"Smartphone", 
             "T":"Tablet",
             "L":"Laptop",
             "V":"VehicleLCD",
             "M":"Monitor"}
status_dict={"F":"Focus"}
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
factor = 360*10



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
fix_gaze = False # GazeAngle1을 CamAngle로 "복사"

base_dir = "/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/"

for date in ["0829", "0830", "0831", "0901", "0902", "0905", "0907", "0908"]:
    wdir = base_dir+f'{date}/processing/S1/'
    sco = Scenario()

    video_list = glob(wdir+"0??/T1/*/RGB/*_rgb_*.mp4")
    video_list.sort()

    for fn in video_list:
    #if True:
        out_dir = fn.split("NIA22EYE")[0]
        fn_ = fn.split("/")[-1]

        cap = cv2.VideoCapture(fn)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print( length )
        sco.set_points(length)

        _, _, ID, _, scenario, device, imgtype, status, action, orientation = fn_.split("_")
        orientation = orientation.split(".mp4")[0]

        #### Eye-tracker
        new_dir = out_dir.replace("RGB/", "Eye-tracker/")
        if not os.path.isdir(new_dir):
            os.mkdir(new_dir)

        # Knee -> lapTop
        #fn_ = fn_.replace("_K_", "_T_")

        fn_out = fn_.replace("rgb", "point")
        fn_out = fn_out.replace(".mp4", ".csv")

        full_fout = new_dir+fn_out
        if os.path.isfile(full_fout) and os.path.getsize(full_fout) > 1000:
            print(">>>>>   file exists", full_fout)
            continue
        with open(full_fout, "w") as f:
            f.write("[point] x y \n")
            px, py = random.choice(sco.scenarios[status])()
            for xx, yy in zip(px, py):
                f.write(f"{int(xx)}, {int(yy)}\n")

            print(full_fout, "done")

    

327
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S03_L_point_A_C_T.csv done
327
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S03_L_point_A_D_T.csv done
327
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S03_L_point_A_E_T.csv done
329
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S03_L_point_A_F_T.csv done
328
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S03_L_point_A_T_T.csv done
327
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S03_L_point_A_U_T.csv done
328
/run/user/1000/gvfs/sftp

327
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S04_L_point_S_E_T.csv done
331
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S04_L_point_S_F_T.csv done
328
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S04_L_point_S_U_T.csv done
328
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S05_L_point_A_C_T.csv done
329
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S05_L_point_A_D_T.csv done
327
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Laptop/Eye-tracker/NIA22EYE_S1_043_T1_S05_L_point_A_E_T.csv done
330
/run/user/1000/gvfs/sftp

300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Monitor/Eye-tracker/NIA22EYE_S1_043_T1_S03_M_point_S_E_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Monitor/Eye-tracker/NIA22EYE_S1_043_T1_S04_M_point_A_C_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Monitor/Eye-tracker/NIA22EYE_S1_043_T1_S04_M_point_A_D_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Monitor/Eye-tracker/NIA22EYE_S1_043_T1_S04_M_point_A_E_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Monitor/Eye-tracker/NIA22EYE_S1_043_T1_S04_M_point_A_S_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Monitor/Eye-tracker/NIA22EYE_S1_043_T1_S04_M_point_A_U_T.csv done
300
/run/user/1000/gvf

309
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_D_E_L.csv done
307
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_D_E_R.csv done
308
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_D_E_T.csv done
304
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_D_F_L.csv done
304
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_D_F_R.csv done
306
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_D_F_T.csv done
308


310
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_N_E_T.csv done
305
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_N_F_L.csv done
306
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_N_F_R.csv done
305
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_N_F_T.csv done
310
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_N_L_L.csv done
311
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0901/processing/S1/043/T1/Smartphone/Eye-tracker/NIA22EYE_S1_043_T1_S02_S_point_N_L_R.csv done
310


...

305
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Smartphone/Eye-tracker/NIA22EYE_S1_055_T1_S04_S_point_N_P_T.csv done
304
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Smartphone/Eye-tracker/NIA22EYE_S1_055_T1_S04_S_point_N_S_L.csv done
306
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Smartphone/Eye-tracker/NIA22EYE_S1_055_T1_S04_S_point_N_S_R.csv done
303
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Smartphone/Eye-tracker/NIA22EYE_S1_055_T1_S04_S_point_N_S_T.csv done
311
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Smartphone/Eye-tracker/NIA22EYE_S1_055_T1_S04_S_point_N_T_L.csv done
306
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Smartphone/Eye-tracker/NIA22EYE_S1_055_T1_S04_S_point_N_T_R.csv done
305


308
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_D_C_T.csv done
301
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_D_D_L.csv done
306
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_D_D_R.csv done
303
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_D_D_T.csv done
310
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_D_E_L.csv done
312
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_D_E_R.csv done
311
/run/user/1000/gvfs/sftp

305
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_F_U_L.csv done
303
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_F_U_R.csv done
304
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_F_U_T.csv done
302
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_N_D_L.csv done
306
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_N_D_R.csv done
303
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S04_T_point_N_D_T.csv done
311
/run/user/1000/gvfs/sftp

302
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S10_T_point_A_U_R.csv done
306
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S10_T_point_A_U_T.csv done
308
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S10_T_point_S_C_L.csv done
310
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S10_T_point_S_C_R.csv done
307
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S10_T_point_S_C_T.csv done
303
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/Tablet/Eye-tracker/NIA22EYE_S1_055_T1_S10_T_point_S_D_L.csv done
307
/run/user/1000/gvfs/sftp

300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_055_T1_S06_V_point_N_E_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_055_T1_S06_V_point_N_U_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_055_T1_S06_V_point_S_C_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_055_T1_S06_V_point_S_D_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_055_T1_S06_V_point_S_E_T.csv done
300
/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/NIA2022/raw/0908/processing/S1/055/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_055_T1_S07_V_point_A_C_T.csv done
300


In [9]:
video_list

[]

In [5]:
new_dir

'/run/user/1000/gvfs/sftp:host=169.254.190.56,port=5002/scratch/processed/0805/processing/S1/008/T1/Laptop/Eye-tracker/'